In [7]:
from datetime import datetime
import requests
import pandas as pd
import csv
from pprint import pprint
from io import StringIO
import statsapi

pd.set_option('display.max_columns', 500)


url = 'http://192.168.1.200/data/daySlate.csv'


response = requests.get(url)
# Check if the request was successful
if response.status_code == 200:
    data = response.content.decode('utf-8')
else:
    print(f"Failed to retrieve data: status code {response.status_code}")
    data = None
if data:
    # Convert string data to StringIO object
    data_io = StringIO(data)
    # Create DataFrame
    df = pd.read_csv(data_io)


In [3]:
df.tail()

,game_id,game_datetime,game_date,game_type,status,away_name,home_name,away_id,home_id,doubleheader,game_num,home_probable_pitcher,away_probable_pitcher,home_pitcher_note,away_pitcher_note,away_score,home_score,current_inning,inning_state,venue_id,venue_name,national_broadcasts,series_status,summary
10,745656,2024-06-05T01:40:00Z,2024-06-04,R,Scheduled,Seattle Mariners,Oakland Athletics,136,133,N,1,Mitch Spence,George Kirby,NaN,NaN,0,0,NaN,NaN,10,Oakland Coliseum,['MLB.tv Free Game'],NaN,2024-06-04 - Seattle Mariners @ Oakland Athlet...
11,746380,2024-06-05T00:10:00Z,2024-06-04,R,Scheduled,St. Louis Cardinals,Houston Astros,138,117,N,1,Spencer Arrighetti,Andre Pallante,NaN,NaN,0,0,NaN,NaN,2392,Minute Maid Park,[],NaN,2024-06-04 - St. Louis Cardinals @ Houston Ast...
12,745568,2024-06-04T22:40:00Z,2024-06-04,R,Scheduled,Milwaukee Brewers,Philadelphia Phillies,158,143,N,1,Cristopher Sánchez,NaN,NaN,NaN,0,0,NaN,NaN,2681,Citizens Bank Park,[],NaN,2024-06-04 - Milwaukee Brewers @ Philadelphia ...
13,746060,2024-06-04T22:40:00Z,2024-06-04,R,Scheduled,Tampa Bay Rays,Miami Marlins,139,146,N,1,Jesús Luzardo,Ryan Pepiot,NaN,NaN,0,0,NaN,NaN,4169,loanDepot park,[],NaN,2024-06-04 - Tampa Bay Rays @ Miami Marlins (S...
14,745739,2024-06-04T23:05:00Z,2024-06-04,R,Scheduled,Minnesota Twins,New York Yankees,142,147,N,1,Luis Gil,Bailey Ober,NaN,NaN,0,0,NaN,NaN,3313,Yankee Stadium,['TBS'],NaN,2024-06-04 - Minnesota Twins @ New York Yankee...


In [4]:
url = 'http://192.168.1.200/data/live_prop_data.csv'


response = requests.get(url)
# Check if the request was successful
if response.status_code == 200:
    data = response.content.decode('utf-8')
else:
    print(f"Failed to retrieve data: status code {response.status_code}")
    data = None
if data:
    # Convert string data to StringIO object
    data_io = StringIO(data)
    # Create DataFrame
    live_df = pd.read_csv(data_io)

In [5]:
live_df

NameError: name 'live_df' is not defined

In [6]:
response = requests.get('https://partner-api.prizepicks.com/projections?')

data = response.json()

In [7]:
new_player_data = []
for item in data['included']:
    if item['type'] == 'new_player':
        new_player_data.append(item)

In [8]:
new_player_data[0]

{'type': 'new_player',
 'id': '193278',
 'attributes': {'combo': False,
  'display_name': 'Ange',
  'image_url': 'https://static.prizepicks.com/images/manual/2AUCwUq.png',
  'league': 'VAL',
  'league_id': 159,
  'market': None,
  'name': 'Ange',
  'position': 'G',
  'team': 'Dough Bros',
  'team_name': None},
 'relationships': {'league': {'data': {'type': 'league', 'id': '159'}}}}

In [9]:

data_df = []
for row in data['data'][:1]:
    player_id = row.get('relationships',{}).get('new_player',{}).get('data',{}).get('id')
    player_info = [d for d in new_player_data if d['id'] == player_id][0]
    player_name = player_info.get('attributes',{}).get('display_name')
    image_url = player_info.get('attributes',{}).get('image_url')

    row_dict = {
        'prop_id' : row.get('id'),
        'league_id' : row.get('relationships',{}).get('league',{}).get('data',{}).get('id'),
        'PP_ID' : player_id,
        'player_name' : player_name,
        'isCombo' :  player_info.get('attributes',{}).get('combo'),
        'league' :  player_info.get('attributes',{}).get('league'),
        'PP league_id' :  player_info.get('attributes',{}).get('league_id'),
        'position' :  player_info.get('attributes',{}).get('position'),
        'Team' :  f"{player_info.get('attributes',{}).get('market')} {player_info.get('attributes',{}).get('team_name')}",
        'is_promo' : row.get('attributes',{}).get('is_promo'),
        'prop_line' : row.get('attributes',{}).get('line_score'),
        'stat_type' : row.get('attributes',{}).get('stat_type'),
        'odds_type' : row.get('attributes',{}).get('odds_type'),
        'projection_type' : row.get('attributes',{}).get('projection_type'),
        'board_time' : row.get('attributes',{}).get('board_time'),
        'start_time' : row.get('attributes',{}).get('start_time').split('T')[0],
        'image_url' : image_url
        
        }
    
    pprint(row)
    data_df.append(row_dict)

data_df = pd.DataFrame(data_df)
data_df = data_df.drop_duplicates(subset=['prop_id'], keep='last')


{'attributes': {'adjusted_odds': None,
                'board_time': '2024-03-12T12:40:00-04:00',
                'custom_image': None,
                'description': 'MIA',
                'end_time': None,
                'flash_sale_line_score': None,
                'game_id': 'dfb531vc2',
                'is_promo': False,
                'line_score': 1375.5,
                'odds_type': 'standard',
                'projection_type': 'Single Stat',
                'rank': 0,
                'refundable': True,
                'start_time': '2024-09-05T20:00:00-04:00',
                'stat_type': 'Receiving Yards',
                'status': 'pre_game',
                'tv_channel': None,
                'updated_at': '2024-05-28T16:19:36-04:00'},
 'id': '2115070',
 'relationships': {'duration': {'data': None},
                   'league': {'data': {'id': '163', 'type': 'league'}},
                   'new_player': {'data': {'id': '39194',
                                          

In [10]:
data_df

,prop_id,league_id,PP_ID,player_name,isCombo,league,PP league_id,position,Team,is_promo,prop_line,stat_type,odds_type,projection_type,board_time,start_time,image_url
0,2115070,163,39194,Tyreek Hill,False,NFLSZN,163,WR,None None,False,1375.5,Receiving Yards,standard,Single Stat,2024-03-12T12:40:00-04:00,2024-09-05,https://static.prizepicks.com/images/players/n...


In [11]:
url = 'http://192.168.1.200/data/all_prop_data.csv'


response = requests.get(url)
# Check if the request was successful
if response.status_code == 200:
    data = response.content.decode('utf-8')
else:
    print(f"Failed to retrieve data: status code {response.status_code}")
    data = None
if data:
    # Convert string data to StringIO object
    data_io = StringIO(data)
    # Create DataFrame
    all_prop_df = pd.read_csv(data_io)

In [12]:
len(all_prop_df)
all_prop_df = all_prop_df.drop_duplicates(['prop_id'], keep='last')

In [49]:
batter_boxscores = pd.read_csv('datasets/complex_batters.csv')
pitcher_boxscores = pd.read_csv('datasets/complex_pitchers.csv')

/var/folders/7f/nl87050s61973ky11lnw6gyr0000gn/T/ipykernel_41061/3923897382.py:1: DtypeWarning: Columns (40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  batter_boxscores = pd.read_csv('datasets/complex_batters.csv')
/var/folders/7f/nl87050s61973ky11lnw6gyr0000gn/T/ipykernel_41061/3923897382.py:2: DtypeWarning: Columns (12,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  pitcher_boxscores = pd.read_csv('datasets/complex_pitchers.csv')


In [14]:
mlbProps = all_prop_df[all_prop_df['league'] == 'MLB']
mlbLiveProps = all_prop_df[all_prop_df['league'] == 'MLBLIVE']

In [45]:
pitcher_boxscores.rename(columns={'Name':'player_name'}, inplace=True)
batter_boxscores.rename(columns={'Name' : 'player_name'}, inplace=True)
pitcher_merged_data = pd.merge(pitcher_boxscores, mlbProps, how='inner', left_on=['player_name', 'game_date'], right_on=['player_name', 'start_time'])
batter_merged_data = pd.merge(batter_boxscores, mlbProps, how='inner', left_on=['player_name', 'game_date'], right_on=['player_name', 'start_time'])

In [16]:

def check_batter_over(row):
    actual_stat = 0
    if row['stat_type'] == 'Total Bases':
        actual_stat = row['TB']
    elif row['stat_type'] == 'Hits+Runs+RBIs':
        actual_stat = row['H+R+RBIs']
    elif row['stat_type'] == 'Hitter Strikeouts':
        actual_stat = row['k']
    elif row['stat_type'] == 'Runs':
        actual_stat = row['r']
    elif row['stat_type'] == 'Hitter Fantasy Score':
        actual_stat = row['Batter Fantasy Score']
    elif row['stat_type'] == 'Stolen Bases':
        actual_stat = row['sb']
    elif row['stat_type'] == 'Home Runs':
        actual_stat = row['hr']
    elif row['stat_type'] == 'Singles':
        actual_stat = row['singles']
    elif row['stat_type'] == 'Hits':
        actual_stat = row['h']
    elif row['stat_type'] == 'Walks':
        actual_stat = row['bb']
    elif row['stat_type'] == 'Doubles':
        actual_stat = row['doubles']
    elif row['stat_type'] == 'RBIs':
        actual_stat = row['rbi']
    return True if actual_stat > row['prop_line'] else False



def check_pitcher_over(row):
    actual_stat = 0
    if row['stat_type'] == 'Pitcher Strikeouts':
        actual_stat = row['k']
    elif row['stat_type'] == 'Walks Allowed':
        actual_stat = row['bb']
    elif row['stat_type'] == 'Hits Allowed':
        actual_stat = row['h']
    elif row['stat_type'] == 'Pitching Outs':
        actual_stat = row['pitching_outs']
    elif row['stat_type'] == 'Pitcher Fantasy Score':
        actual_stat = row['Pitcher Fantasy Score']
    return True if actual_stat > row['prop_line'] else False
pitcher_merged_data['Over_Under'] = pitcher_merged_data.apply(check_pitcher_over, axis=1)
batter_merged_data['Over_Under'] = batter_merged_data.apply(check_batter_over, axis = 1)



pitcher_prop_info = pitcher_merged_data[pitcher_merged_data['position'] == 'P'][['Over_Under', 'player_name', 'Team_x', 'ip','k', 'bb', 'er', 'h', 'PitcherIsWinBool', 'isStarter','pitch_count_MA3', 'strikeout_MA3', 'walks_MA3', 'h_MA3', 'isWinner','Pitcher Fantasy Score', 'prop_line', 'odds_type','game_date', 'Opponent', 'stat_type', 'image_url']].sort_values(by='game_date')
pitcher_prop_info.rename(columns={'Team_x': 'Team'})

batter_prop_info = batter_merged_data[batter_merged_data['position_y'] != 'P'][['Over_Under', 'player_name', 'k', 'bb', 'r', 'h', 'Batter Fantasy Score', 'doubles','hr','sb', 'H+R+RBIs','isStarter', 'Team_x', 'game_date', 'prop_line', 'stat_type', 'odds_type', 'image_url']].sort_values(by='game_date')
batter_prop_info.rename(columns={'Team_x': 'Team'})


,Over_Under,player_name,k,bb,r,h,Batter Fantasy Score,doubles,hr,sb,H+R+RBIs,isStarter,Team,game_date,prop_line,stat_type,odds_type,image_url
450,False,Brandon Lowe,1,1,0,0,2,0,0,0,0,True,Tampa Bay Rays,2024-05-22,1.5,Hits+Runs+RBIs,standard,https://static.prizepicks.com/images/players/m...
198,True,Matt Olson,2,0,1,2,17,0,1,0,4,True,Atlanta Braves,2024-05-22,0.5,Runs,standard,https://static.prizepicks.com/images/players/m...
199,True,Matt Olson,2,0,1,2,17,0,1,0,4,True,Atlanta Braves,2024-05-22,7.0,Hitter Fantasy Score,standard,https://static.prizepicks.com/images/players/m...
200,True,Orlando Arcia,2,0,1,1,7,1,0,0,2,True,Atlanta Braves,2024-05-22,1.5,Hits+Runs+RBIs,standard,https://static.prizepicks.com/images/players/m...
201,True,Michael Harris II,0,0,2,2,12,1,0,0,4,True,Atlanta Braves,2024-05-22,1.5,Hits+Runs+RBIs,standard,https://static.prizepicks.com/images/players/m...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24411,True,Marcus Semien,1,1,1,2,19,0,1,0,4,True,Texas Rangers,2024-06-03,0.5,Walks,demon,https://static.prizepicks.com/images/players/m...
24410,True,Marcus Semien,1,1,1,2,19,0,1,0,4,True,Texas Rangers,2024-06-03,0.5,Hitter Strikeouts,goblin,https://static.prizepicks.com/images/players/m...
24409,False,Marcus Semien,1,1,1,2,19,0,1,0,4,True,Texas Rangers,2024-06-03,0.5,Doubles,demon,https://static.prizepicks.com/images/players/m...
24407,True,Jake Rogers,0,0,2,2,28,0,2,0,6,True,Detroit Tigers,2024-06-03,1.5,Total Bases,demon,https://static.prizepicks.com/images/players/m...


In [62]:
pitcher_boxscores = pd.read_csv('datasets/complex_pitchers.csv')

pitcher_boxscores['game_datetime'] = pd.to_datetime(pitcher_boxscores['game_datetime'])
pitcher_boxscores['starting_time'] = pitcher_boxscores['game_datetime'].dt.time



In [63]:
#pitcher_boxscores['starting_time'] = pitcher_boxscores['game_datetime']
pitcher_boxscores

,Team,team_id,namefield,ip,h,r,er,bb,k,hr,p,s,era,name,personId,note,game_id,date,Name,game_datetime,game_date,game_type,status,away_name,home_name,away_id,home_id,doubleheader,game_num,home_probable_pitcher,away_probable_pitcher,home_pitcher_note,away_pitcher_note,away_score,home_score,current_inning,inning_state,venue_id,venue_name,national_broadcasts,series_status,winning_team,losing_team,winning_pitcher,losing_pitcher,save_pitcher,summary,losing_Team,isWinner,Opponent,isStarter,pitcherIsWinner,pitch_count_MA3,strikeout_MA3,walks_MA3,h_MA3,pitching_outs,seasonNumber,game_score,performance_category,PitcherIsWinBool,Pitcher Fantasy Score,bullpen_ERA,bullpen_WHIP,days_between_games,starting_time
0,New York Mets,121,"Colon (W, 1-0)",6.0,3,1,1,1,8,1,86,59,1.50,Colon,112526,"(W, 1-0)",413657,2015-04-06,Bartolo Colon,2015-04-06 20:05:00+00:00,2015-04-06,R,Final,New York Mets,Washington Nationals,121,120,N,1,Max Scherzer,Bartolo Colon,It will be the first Opening Day start of Sche...,"At 41 years, 317 days old, Colon is set to bec...",3,1,9.0,Bottom,3309,Nationals Park,['ESPN'],NYM leads 1-0,New York Mets,Washington Nationals,Bartolo Colon,Max Scherzer,Buddy Carlyle,2015-04-06 - New York Mets (3) @ Washington Na...,NaN,True,Washington Nationals,True,True,NaN,NaN,NaN,NaN,18,2015,73,Good,1,49,0.000,0.000,NaN,20:05:00
1,New York Mets,121,"Colon (W, 2-0)",7.0,6,3,3,0,5,1,77,60,2.77,Colon,112526,"(W, 2-0)",413733,2015-04-12,Bartolo Colon,2015-04-12 17:35:00+00:00,2015-04-12,R,Final,New York Mets,Atlanta Braves,121,144,N,1,Alex Wood,Bartolo Colon,Wood delivered five strong innings while provi...,Though the Mets received plenty of criticism f...,4,3,9.0,Bottom,16,Turner Field,[],ATL wins 2-1,New York Mets,Atlanta Braves,Bartolo Colon,Juan Jaime,Jeurys Familia,2015-04-12 - New York Mets (4) @ Atlanta Brave...,NaN,True,Atlanta Braves,True,True,NaN,NaN,NaN,NaN,21,2015,66,Good,1,37,0.000,0.769,6.0,17:35:00
2,New York Mets,121,"Colon (W, 3-0)",7.0,6,1,1,0,5,1,91,66,2.25,Colon,112526,"(W, 3-0)",413797,2015-04-17,Bartolo Colon,2015-04-17 23:10:00+00:00,2015-04-17,R,Final,Miami Marlins,New York Mets,146,121,N,1,Bartolo Colon,David Phelps,Though Colon delivered another quality start i...,Phelps is getting the first crack at filling t...,1,4,9.0,Top,3289,Citi Field,[],NYM leads 2-0,New York Mets,Miami Marlins,Bartolo Colon,Brad Hand,Jeurys Familia,2015-04-17 - Miami Marlins (1) @ New York Mets...,NaN,True,Miami Marlins,True,True,NaN,NaN,NaN,NaN,21,2015,72,Good,1,43,0.000,0.500,5.0,23:10:00
3,New York Mets,121,"Colon (W, 4-0)",6.0,7,3,3,0,5,0,91,59,2.77,Colon,112526,"(W, 4-0)",413873,2015-04-23,Bartolo Colon,2015-04-23 17:10:00+00:00,2015-04-23,R,Final,Atlanta Braves,New York Mets,144,121,N,1,Bartolo Colon,Julio Teheran,Colon has won all three times out and twice co...,Teheran was tagged for a career-high four home...,3,6,9.0,Top,3289,Citi Field,[],NYM wins 3-0,New York Mets,Atlanta Braves,Bartolo Colon,Julio Teheran,Jeurys Familia,2015-04-23 - Atlanta Braves (3) @ New York Met...,NaN,True,Atlanta Braves,True,True,84.67,6.00,0.33,5.00,18,2015,58,Average,1,34,0.000,0.667,6.0,17:10:00
4,New York Mets,121,"Colon (L, 4-1)",6.2,9,4,4,0,2,1,91,64,3.31,Colon,112526,"(L, 4-1)",413966,2015-04-29,Bartolo Colon,2015-04-29 23:10:00+00:00,2015-04-29,R,Final,New York Mets,Miami Marlins,121,146,N,1,Mat Latos,Bartolo Colon,"Latos comes off his best outing, going a seaso...",Colon is just the fourth pitcher aged 40 or ol...,3,7,9.0,Top,4169,Marlins Park,[],MIA wins 2-1,Miami Marlins,New York Mets,Sam Dyson,Bartolo Colon,NaN,2015-04-29 - New York Mets (3) @ Miami Marlins...,NaN,False,Miami Marlins,True,False,84.67,6.00,0.33,5.33,20,2015,52,Average,0,14,67.500,10.000,6.0,23:10:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177652,Los Angeles Dodger

In [56]:
bullpen_boxscores = pitcher_boxscores[pitcher_boxscores['isStarter'] == False][['Team', 'team_id', 'ip', 'h', 'r', 'er', 'bb', 'k', 'hr','p', 's','era','personId', 'game_id', 'date','Name', 'game_date','away_name', 'home_name', 'away_id', 'home_id', 'doubleheader', 'home_probable_pitcher', 'away_probable_pitcher', 'away_score', 'home_score', 'winning_team', 'losing_team', 'winning_pitcher', 'losing_pitcher', 'save_pitcher', 'isWinner', 'pitching_outs', 'game_score', 'PitcherIsWinBool']]

grouped = bullpen_boxscores.groupby(['Team', 'game_id']).agg({'er':'sum', 'ip':'sum', 'bb':'sum', 'h': 'sum'}).reset_index()

grouped['bullpen_WHIP'] = round((grouped['bb'] + grouped['h']) / grouped['ip'],3)
grouped['bullpen_ERA'] = round((grouped['er'] / grouped['ip']) * 9,3)
pitcher_boxscores = pitcher_boxscores.merge(grouped[['Team', 'game_id', 'bullpen_ERA', 'bullpen_WHIP']], on=['Team', 'game_id'])

pitcher_boxscores['game_date'] = pd.to_datetime(pitcher_boxscores['game_date'])


pitcher_boxscores = pitcher_boxscores.sort_values(by=['personId', 'game_date'])
pitcher_boxscores['days_between_games'] = pitcher_boxscores.groupby(['personId','seasonNumber'])['game_date'].diff().dt.days


pitcher_boxscores

,Team,team_id,namefield,ip,h,r,er,bb,k,hr,p,s,era,name,personId,note,game_id,date,Name,game_datetime,game_date,game_type,status,away_name,home_name,away_id,home_id,doubleheader,game_num,home_probable_pitcher,away_probable_pitcher,home_pitcher_note,away_pitcher_note,away_score,home_score,current_inning,inning_state,venue_id,venue_name,national_broadcasts,series_status,winning_team,losing_team,winning_pitcher,losing_pitcher,save_pitcher,summary,losing_Team,isWinner,Opponent,isStarter,pitcherIsWinner,pitch_count_MA3,strikeout_MA3,walks_MA3,h_MA3,pitching_outs,seasonNumber,game_score,performance_category,PitcherIsWinBool,Pitcher Fantasy Score,bullpen_ERA,bullpen_WHIP,days_between_games
496,New York Mets,121,"Colon (W, 1-0)",6.0,3,1,1,1,8,1,86,59,1.50,Colon,112526,"(W, 1-0)",413657,2015-04-06,Bartolo Colon,2015-04-06T20:05:00Z,2015-04-06,R,Final,New York Mets,Washington Nationals,121,120,N,1,Max Scherzer,Bartolo Colon,It will be the first Opening Day start of Sche...,"At 41 years, 317 days old, Colon is set to bec...",3,1,9.0,Bottom,3309,Nationals Park,['ESPN'],NYM leads 1-0,New York Mets,Washington Nationals,Bartolo Colon,Max Scherzer,Buddy Carlyle,2015-04-06 - New York Mets (3) @ Washington Na...,NaN,True,Washington Nationals,True,True,NaN,NaN,NaN,NaN,18,2015,73,Good,1,49,0.000,0.000,NaN
216,New York Mets,121,"Colon (W, 2-0)",7.0,6,3,3,0,5,1,77,60,2.77,Colon,112526,"(W, 2-0)",413733,2015-04-12,Bartolo Colon,2015-04-12T17:35:00Z,2015-04-12,R,Final,New York Mets,Atlanta Braves,121,144,N,1,Alex Wood,Bartolo Colon,Wood delivered five strong innings while provi...,Though the Mets received plenty of criticism f...,4,3,9.0,Bottom,16,Turner Field,[],ATL wins 2-1,New York Mets,Atlanta Braves,Bartolo Colon,Juan Jaime,Jeurys Familia,2015-04-12 - New York Mets (4) @ Atlanta Brave...,NaN,True,Atlanta Braves,True,True,NaN,NaN,NaN,NaN,21,2015,66,Good,1,37,0.000,0.769,6.0
50,New York Mets,121,"Colon (W, 3-0)",7.0,6,1,1,0,5,1,91,66,2.25,Colon,112526,"(W, 3-0)",413797,2015-04-17,Bartolo Colon,2015-04-17T23:10:00Z,2015-04-17,R,Final,Miami Marlins,New York Mets,146,121,N,1,Bartolo Colon,David Phelps,Though Colon delivered another quality start i...,Phelps is getting the first crack at filling t...,1,4,9.0,Top,3289,Citi Field,[],NYM leads 2-0,New York Mets,Miami Marlins,Bartolo Colon,Brad Hand,Jeurys Familia,2015-04-17 - Miami Marlins (1) @ New York Mets...,NaN,True,Miami Marlins,True,True,NaN,NaN,NaN,NaN,21,2015,72,Good,1,43,0.000,0.500,5.0
1357,New York Mets,121,"Colon (W, 4-0)",6.0,7,3,3,0,5,0,91,59,2.77,Colon,112526,"(W, 4-0)",413873,2015-04-23,Bartolo Colon,2015-04-23T17:10:00Z,2015-04-23,R,Final,Atlanta Braves,New York Mets,144,121,N,1,Bartolo Colon,Julio Teheran,Colon has won all three times out and twice co...,Teheran was tagged for a career-high four home...,3,6,9.0,Top,3289,Citi Field,[],NYM wins 3-0,New York Mets,Atlanta Braves,Bartolo Colon,Julio Teheran,Jeurys Familia,2015-04-23 - Atlanta Braves (3) @ New York Met...,NaN,True,Atlanta Braves,True,True,84.67,6.00,0.33,5.00,18,2015,58,Average,1,34,0.000,0.667,6.0
2412,New York Mets,121,"Colon (L, 4-1)",6.2,9,4,4,0,2,1,91,64,3.31,Colon,112526,"(L, 4-1)",413966,2015-04-29,Bartolo Colon,2015-04-29T23:10:00Z,2015-04-29,R,Final,New York Mets,Miami Marlins,121,146,N,1,Mat Latos,Bartolo Colon,"Latos comes off his best outing, going a seaso...",Colon is just the fourth pitcher aged 40 or ol...,3,7,9.0,Top,4169,Marlins Park,[],MIA wins 2-1,Miami Marlins,New York Mets,Sam Dyson,Bartolo Colon,NaN,2015-04-29 - New York Mets (3) @ Miami Marlins...,NaN,False,Miami Marlins,True,False,84.67,6.00,0.33,5.33,20,2015,52,Average,0,14,67.500,10.000,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174658,Los Angeles Dodgers,119,"Yamamoto (W, 4-1)",8.0,5,2,2,0,5,2,97,73,2.79,Yamamoto,808967,"(W, 4-1

In [47]:
grouped

,Team,game_id,er,ip,bb,h,bullpen_WHIP,bullpen_ERA
0,Arizona Diamondbacks,413660,0,3.4,1,3,1.176,0.000
1,Arizona Diamondbacks,413669,0,3.2,1,2,0.938,0.000
2,Arizona Diamondbacks,413682,2,4.2,0,5,1.190,4.286
3,Arizona Diamondbacks,413701,0,5.0,0,3,0.600,0.000
4,Arizona Diamondbacks,413716,0,3.0,0,1,0.333,0.000
...,...,...,...,...,...,...,...,...
41072,Washington Nationals,746965,0,2.0,0,1,0.500,0.000
41073,Washington Nationals,747114,0,3.1,1,1,0.645,0.000
41074,Washington Nationals,747115,2,2.0,1,3,2.000,9.000
41075,Washington Nationals,747117,0,2.5,0,2,0.800,0.000


In [4]:
basic_gamelogs = pd.read_csv('datasets/basicGameLogs.csv')

In [5]:
basic_gamelogs.head()

,game_id,game_datetime,game_date,game_type,status,away_name,home_name,away_id,home_id,doubleheader,game_num,home_probable_pitcher,away_probable_pitcher,home_pitcher_note,away_pitcher_note,away_score,home_score,current_inning,inning_state,venue_id,venue_name,national_broadcasts,series_status,winning_team,losing_team,winning_pitcher,losing_pitcher,save_pitcher,summary,losing_Team
0,429641,2015-03-10T20:05:00Z,2015-03-10,S,Final,Chicago Cubs,Cleveland Indians,112,114,N,1,Trevor Bauer,Travis Wood,NaN,NaN,6,10,9.0,Top,3834,Goodyear Ballpark,['MLBN (delay)'],CLE wins Spring,Cleveland Indians,Chicago Cubs,Shaun Marcum,Blake Parker,NaN,2015-03-10 - Chicago Cubs (6) @ Cleveland Indi...,NaN
1,429879,2015-03-10T20:05:00Z,2015-03-10,S,Final,San Francisco Giants,San Diego Padres,137,135,N,1,Ian Kennedy,Yusmeiro Petit,NaN,NaN,5,10,9.0,Top,2530,Peoria Stadium,[],SD wins Spring,San Diego Padres,San Francisco Giants,Jay Jackson,Yusmeiro Petit,NaN,2015-03-10 - San Francisco Giants (5) @ San Di...,NaN
2,429579,2015-03-10T17:05:00Z,2015-03-10,S,Final,Tampa Bay Rays,Boston Red Sox,139,111,N,1,Justin Masterson,Matt Andriese,NaN,NaN,1,5,9.0,Top,4309,JetBlue Park,[],BOS wins Spring,Boston Red Sox,Tampa Bay Rays,Justin Masterson,Matt Andriese,NaN,2015-03-10 - Tampa Bay Rays (1) @ Boston Red S...,NaN
3,429832,2015-03-10T20:05:00Z,2015-03-10,S,Final,Arizona Diamondbacks,Oakland Athletics,109,133,N,1,Sonny Gray,Trevor Cahill,NaN,NaN,5,6,9.0,Top,2507,Hohokam Stadium,[],OAK wins Spring,Oakland Athletics,Arizona Diamondbacks,Eric O'Flaherty,Archie Bradley,Ryan Cook,2015-03-10 - Arizona Diamondbacks (5) @ Oaklan...,NaN
4,429658,2015-03-10T20:10:00Z,2015-03-10,S,Final,Seattle Mariners,Colorado Rockies,136,115,N,1,Kyle Kendrick,Félix Hernández,NaN,NaN,3,9,9.0,Top,4249,Salt River Fields at Talking Stick,[],COL leads 1-0,Colorado Rockies,Seattle Mariners,LaTroy Hawkins,Dominic Leone,NaN,2015-03-10 - Seattle Mariners (3) @ Colorado R...,NaN


In [10]:
daySlate = []
current_date = datetime.now().strftime('%Y-%m-%d')
# current_date = datetime.now() + timedelta(days=1)

sched = statsapi.schedule(date=current_date,team=136)




In [11]:
sched

[{'game_id': 745246,
  'game_datetime': '2024-06-13T01:40:00Z',
  'game_date': '2024-06-12',
  'game_type': 'R',
  'status': 'Pre-Game',
  'away_name': 'Chicago White Sox',
  'home_name': 'Seattle Mariners',
  'away_id': 145,
  'home_id': 136,
  'doubleheader': 'N',
  'game_num': 1,
  'home_probable_pitcher': 'Bryce Miller',
  'away_probable_pitcher': 'Jonathan Cannon',
  'home_pitcher_note': '',
  'away_pitcher_note': '',
  'away_score': 0,
  'home_score': 0,
  'current_inning': 1,
  'inning_state': 'Top',
  'venue_id': 680,
  'venue_name': 'T-Mobile Park',
  'national_broadcasts': ['MLBN (out-of-market only)'],
  'series_status': 'SEA leads 2-0',
  'summary': '2024-06-12 - Chicago White Sox @ Seattle Mariners (Pre-Game)'}]

In [12]:
statsapi.boxscore_data(745246)

{'gameId': '2024/06/12/chamlb-seamlb-1',
 'teamInfo': {'away': {'id': 145,
   'abbreviation': 'CWS',
   'teamName': 'White Sox',
   'shortName': 'Chi White Sox'},
  'home': {'id': 136,
   'abbreviation': 'SEA',
   'teamName': 'Mariners',
   'shortName': 'Seattle'}},
 'playerInfo': {'ID672820': {'id': 672820,
   'fullName': 'Lenyn Sosa',
   'boxscoreName': 'Sosa, L'},
  'ID455117': {'id': 455117,
   'fullName': 'Martín Maldonado',
   'boxscoreName': 'Maldonado, M'},
  'ID657508': {'id': 657508,
   'fullName': 'Mike Baumann',
   'boxscoreName': 'Baumann'},
  'ID682988': {'id': 682988,
   'fullName': 'Tyler Locklear',
   'boxscoreName': 'Locklear'},
  'ID641487': {'id': 641487,
   'fullName': 'J.P. Crawford',
   'boxscoreName': 'Crawford, J'},
  'ID686676': {'id': 686676,
   'fullName': 'Korey Lee',
   'boxscoreName': 'Lee, K'},
  'ID605154': {'id': 605154,
   'fullName': 'John Brebbia',
   'boxscoreName': 'Brebbia'},
  'ID676113': {'id': 676113,
   'fullName': 'Zach DeLoach',
   'boxscor